In [1]:
import sys,os
project_root = os.path.dirname(os.getcwd())
sys.path.insert(0,project_root)

from src.data_handling.load_data import *
from src.data_handling.preprocessing import *
from src.models.model_setup import *
from src.trainer.training import *
from src.trainer.file_utils import *
from transformers import set_seed

c:\Users\Hector Auvinen\Documents\GitHub\adapter_experiments\adapter_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

seed_value = 42
set_seed(seed_value)

# supported tasks:
# commonsense_qa: csqa # should this be classification?
# cosmos_qa: cosqa # should this be classification?
# social_i_qa: sociqa
# hellaswag: hswag
# winogrande: wgrande


In [3]:
task = "cosmos_qa"
model_name = "bert-base-uncased"
output_dir = "C:/Users/Hector Auvinen/Desktop/eval_results"

In [4]:
data = load_hf_dataset(task,debug=False)

tokenizer = get_tokenizer(model_name)

encode = get_encoding(task)

using bert tokenizer
getting encoding:
<function encode_cosmosqa at 0x000001BC88F6DE50>


In [5]:
dataset = preprocess_dataset(data,encode,tokenizer)

mapped
{'train': ['id', 'context', 'question', 'answer0', 'answer1', 'answer2', 'answer3', 'label', 'input_ids', 'attention_mask'], 'test': ['id', 'context', 'question', 'answer0', 'answer1', 'answer2', 'answer3', 'label', 'input_ids', 'attention_mask'], 'validation': ['id', 'context', 'question', 'answer0', 'answer1', 'answer2', 'answer3', 'label', 'input_ids', 'attention_mask']}


In [6]:
num_labels = get_label_count(dataset)
print("labels",num_labels)
model = setup_model(model_name,num_labels,dataset)

adapter_config = adapters.BnConfig(
                          output_adapter=True,
                          mh_adapter=False,
                          reduction_factor=2,
                          non_linearity="relu")

model = add_mc_adapter(task_name=task,model=model,num_labels=num_labels,adapter_config=adapter_config)

default_args = TrainingParameters(per_device_train_batch_size=2)
default_args.lr_scheduler_type = "linear"
#default_args.per_device_eval_batch_size = 4
print(default_args.per_device_train_batch_size)

labels 4


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2


In [7]:
train_args = get_training_arguments(default_args)

trainer = get_trainer(train_args,dataset,model,early_stopping=3)

trainer.train()

                                                      
  0%|          | 50/378930 [05:52<49:22:05,  2.13it/s]

{'eval_loss': 1.352368950843811, 'eval_accuracy': 0.2984924623115578, 'eval_runtime': 326.7183, 'eval_samples_per_second': 9.136, 'eval_steps_per_second': 1.145, 'epoch': 0.0}


                                                         
  0%|          | 100/378930 [11:43<48:45:07,  2.16it/s]

{'eval_loss': 1.463514804840088, 'eval_accuracy': 0.288107202680067, 'eval_runtime': 327.5285, 'eval_samples_per_second': 9.114, 'eval_steps_per_second': 1.142, 'epoch': 0.01}


                                                          
  0%|          | 150/378930 [17:38<51:18:33,  2.05it/s]

...

                                                              
  0%|          | 1100/378930 [2:11:44<47:25:58,  2.21it/s]

{'eval_loss': 1.2766929864883423, 'eval_accuracy': 0.46968174204355106, 'eval_runtime': 326.8544, 'eval_samples_per_second': 9.133, 'eval_steps_per_second': 1.144, 'epoch': 0.09}


                                                             
  0%|          | 1150/378930 [2:17:35<753:18:09,  7.18s/it]

{'eval_loss': 1.2511285543441772, 'eval_accuracy': 0.46164154103852595, 'eval_runtime': 326.8384, 'eval_samples_per_second': 9.133, 'eval_steps_per_second': 1.144, 'epoch': 0.09}
{'train_runtime': 8255.2616, 'train_samples_per_second': 91.803, 'train_steps_per_second': 45.902, 'train_loss': 1.3221454255477243, 'epoch': 0.09}


TrainOutput(global_step=1150, training_loss=1.3221454255477243, metrics={'train_runtime': 8255.2616, 'train_samples_per_second': 91.803, 'train_steps_per_second': 45.902, 'train_loss': 1.3221454255477243, 'epoch': 0.09})

In [8]:
eval_results = trainer.evaluate()

100%|██████████| 374/374 [05:40<00:00,  1.10it/s]


In [9]:
write_eval_results(eval_results,output_dir,task,trainer,adapter_config)

Writing eval results
{'eval_loss': 1.2569043636322021, 'eval_accuracy': 0.46968174204355106, 'eval_runtime': 341.0548, 'eval_samples_per_second': 8.752, 'eval_steps_per_second': 1.097, 'epoch': 0.09}
